In [2]:
import tkinter as tk
from PIL import Image,ImageTk
import string
from tkinter import scrolledtext 
from sklearn import model_selection, preprocessing ,linear_model   #,naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import nltk
from nltk.corpus import stopwords
import pandas as pd
from snowballstemmer import TurkishStemmer
turkStem=TurkishStemmer()

form = tk.Tk()
form.geometry("1000x600")
form.title("Proje")
form.configure(background="black")
form.resizable(False,False) #Arkaplan sabit olur.

data = pd.read_csv('ÖnİşlenmişVeriler2.csv',encoding = "utf-8")
df = pd.DataFrame() 
df["text"] = data["text"]
df["label"] = data["label"]
    
train_x ,test_x,train_y,test_y = model_selection.train_test_split(df["text"],df["label"])
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)
vectorizer = CountVectorizer()
vectorizer.fit(train_x)
X_train_count = vectorizer.transform(train_x)
X_test_count = vectorizer.transform(test_x)
    
loj = linear_model.LogisticRegression()
loj_model = loj.fit(X_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           X_test_count, 
                                           test_y, 
                                           cv = 10).mean()


def MakineOgrenmesi(yazı1): 

    #print("TF-IDF Doğruluk Oranı:", accuracy)
    yazı1 = yazı1.lower() #Yazının küçük harflere çevrilmesi
    yazı = ""
    for i in yazı1: #Yazının noktalama işaretlerinden ayrılması
        if i not in string.punctuation:
            yazı += i

    yazı = yazı.split(" ")
    #Yazının köklerine ayrılması:
    for i in range(0,len(yazı)):
        yazı[i] = turkStem.stemWord(yazı[i])
    kelime = ""
    for i in range(0,len(yazı)):
        kelime += yazı[i] + " "
    kelime = kelime.strip()
    #print(kelime)
    kelime = [kelime]
    yeni_yorum = kelime
    v = CountVectorizer()
    v.fit(train_x)
    yeni_yorum = v.transform(yeni_yorum)
    #print(loj_model.predict(yeni_yorum)[0])
    return(loj_model.predict(yeni_yorum)[0])
    

def yazdır():    
    from_scrolledtext = TestEdilecekYazı.get('1.0', 'end')
    #print(type(from_scrolledtext))        
    deger = MakineOgrenmesi(from_scrolledtext)
    if(deger == 1):
        SonucYazıLabel["text"] = "Pozitif bir yazı girdiniz"
    else:
        SonucYazıLabel["text"] = "Negatif bir yazı girdiniz"
        

resim = Image.open("Arkaplan.jpg")
resim = ImageTk.PhotoImage(resim)
label = tk.Label(form,image = resim)
label.pack()
    
#Duygu Tahmin etmek.
baslık = tk.Label(form,text = "Haber Duygu Tahmin Projesi",font = "verdana 20 bold",bg = "light blue")
baslık.place(x=300,y=50)

TestEdilecekYazıLabel = tk.Label(form,text = "Test Edilmesini İstediğiniz Cümleyi Girin.",font = "verdana 12",bg = "light blue")
TestEdilecekYazıLabel.place(x=80,y=120)

TestEdilecekYazı = scrolledtext.ScrolledText(form,height= 9,width=50,wrap = tk.WORD,font = ("Times New Roman",12))
TestEdilecekYazı.place(x = 100, y = 150)
TestEdilecekYazı.insert("1.0","")

TestEtButonu = tk.Button(form,text = "Test Et",command = yazdır,height= 4, width=20)
TestEtButonu.place(x = 600 ,y = 205)

YazıLabel = tk.Label(form,text = "Girdiğiniz cümlenin duygu değeri:",font = "verdana 12",bg = "light blue")
YazıLabel.place(x=100,y=420)

SonucYazıLabel = tk.Label(form,font = "verdana 12",bg = "light blue")
SonucYazıLabel.place(x=400,y=420)

"""
ArkaplanDegistir = tk.Button(form,text = "Arka planı degistir",command = arkaplan, height= 3, width=16)
ArkaplanDegistir.place(x = 600 ,y = 225)
"""
form.mainloop()